# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [34]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [73]:
# enter your code here
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders[20:30]

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
20,20,536367,48187,2010,12,3,8,doormat new england,4,2010-12-01 08:34:00,7.95,13047,United Kingdom,31.80
21,21,536368,22960,2010,12,3,8,jam making set with jars,6,2010-12-01 08:34:00,4.25,13047,United Kingdom,25.50
22,22,536368,22913,2010,12,3,8,red coat rack paris fashion,3,2010-12-01 08:34:00,4.95,13047,United Kingdom,14.85
23,23,536368,22912,2010,12,3,8,yellow coat rack paris fashion,3,2010-12-01 08:34:00,4.95,13047,United Kingdom,14.85
24,24,536368,22914,2010,12,3,8,blue coat rack paris fashion,3,2010-12-01 08:34:00,4.95,13047,United Kingdom,14.85
25,25,536369,21756,2010,12,3,8,bath building block word,3,2010-12-01 08:35:00,5.95,13047,United Kingdom,17.85
26,26,536370,22728,2010,12,3,8,alarm clock bakelike pink,24,2010-12-01 08:45:00,3.75,12583,France,90.00
27,27,536370,22727,2010,12,3,8,alarm clock bakelike red,24,2010-12-01 08:45:00,3.75,12583,France,90.00
28,28,536370,22726,2010,12,3,8,alarm clock bakelike green,12,2010-12-01 08:45:00,3.75,12583,France,45.00
29,29,536370,21724,2010,12,3,8,panda and bunnies sticker sheet,12,2010-12-01 08:45:00,0.85,12583,France,10.20


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [117]:
#aggregate the  amount_spent for unique customers. VIP customers
orders.groupby(['CustomerID', 'Country'])['amount_spent'].agg(['sum', 'mean', 'std', 'median']).sort_values(by='sum', ascending=False).head(10)

,,sum,mean,std,median
CustomerID,Country,,,,
14646,Netherlands,280206.02,134.714433,170.033502,105.60
18102,United Kingdom,259657.30,602.453132,691.620535,394.80
17450,United Kingdom,194550.79,577.302047,819.507014,288.00
16446,United Kingdom,168472.50,56157.500000,97265.131753,1.65
14911,EIRE,143825.06,25.334694,44.511676,17.00
12415,Australia,124914.53,174.461634,181.084818,135.00
14156,EIRE,117379.63,83.842593,159.765633,21.45
17511,United Kingdom,91062.38,94.561142,82.042770,75.60
16029,United Kingdom,81024.84,334.813388,614.157436,174.00


In [124]:
aggregated_sum=orders.pivot_table(index=["CustomerID", "Country"], values=["amount_spent"], aggfunc=('sum')).sort_values(by='amount_spent', ascending=False)

In [125]:
aggregated_sum.head(10)

,,amount_spent
CustomerID,Country,
14646,Netherlands,280206.02
18102,United Kingdom,259657.30
17450,United Kingdom,194550.79
16446,United Kingdom,168472.50
14911,EIRE,143825.06
12415,Australia,124914.53
14156,EIRE,117379.63
17511,United Kingdom,91062.38
16029,United Kingdom,81024.84


In [105]:
orders.pivot_table(index=["year"], values=["amount_spent"], aggfunc='sum')
#sum of amount spent per year : method 1

,amount_spent
year,
2010,5.727139e+05
2011,8.338694e+06


In [106]:
orders.groupby(['year'])['amount_spent'].sum()
#sum of amount spent per year : method 2

year
2010    5.727139e+05
2011    8.338694e+06
Name: amount_spent, dtype: float64

In [107]:
orders['amount_spent'].sum()
#sum of amount spent for two years

8911407.904

SUB Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?

In [108]:
orders['amount_spent'].quantile([.75, 0.95])
# only the quantile per one order/row

0.75    19.8
0.95    67.5
Name: amount_spent, dtype: float64

In [127]:
aggregated_sum.amount_spent.quantile([0.5, .75, 0.95])
# the quantile per sum

0.50     671.960
0.75    1661.195
0.95    5774.278
Name: amount_spent, dtype: float64

In [128]:
aggregated_sum['Quantile_rank']=pd.qcut(aggregated_sum['amount_spent'],4,labels=False)

In [138]:
aggregated_sum.head(10)
# preffered customer with quantile rank 3 

,,amount_spent,Quantile_rank
CustomerID,Country,,
14646,Netherlands,280206.02,3
18102,United Kingdom,259657.30,3
17450,United Kingdom,194550.79,3
16446,United Kingdom,168472.50,3
14911,EIRE,143825.06,3
12415,Australia,124914.53,3
14156,EIRE,117379.63,3
17511,United Kingdom,91062.38,3
16029,United Kingdom,81024.84,3


In [136]:
preffered=aggregated_sum[aggregated_sum['amount_spent'] > aggregated_sum['amount_spent'].quantile(.95)]

In [137]:
preffered
# preffered customer with quantile rank over 0.95 

,,amount_spent,Quantile_rank
CustomerID,Country,,
14646,Netherlands,280206.02,3
18102,United Kingdom,259657.30,3
17450,United Kingdom,194550.79,3
16446,United Kingdom,168472.50,3
14911,EIRE,143825.06,3
12415,Australia,124914.53,3
14156,EIRE,117379.63,3
17511,United Kingdom,91062.38,3
16029,United Kingdom,81024.84,3


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [140]:
aggregated_sum_country=orders.pivot_table(index=["Country"], values=["amount_spent"], aggfunc=('sum')).sort_values(by='amount_spent', ascending=False)

In [141]:
aggregated_sum_country
# by total  amount spent, but not by numner of  VIP customers

,amount_spent
Country,
United Kingdom,7.308392e+06
Netherlands,2.854463e+05
EIRE,2.655459e+05
Germany,2.288671e+05
France,2.090240e+05
Australia,1.385213e+05
Spain,6.157711e+04
Switzerland,5.644395e+04
Belgium,4.119634e+04
